# Introduction

Hello, I'm **Wesley**, nice to meet you ! 👋

![anchor-man.jpeg](attachment:b5c8cbf4-7178-450d-94db-74dbcc2ed6e2.jpeg)

**BREAKING: We are making a news topic classifier!**

Here we'll using transfer learning on [BERT](https://huggingface.co/distilbert-base-uncased) with the [AG News Dataset](https://huggingface.co/datasets/ag_news).

To try out the model without running the notebook, check out my [HuggingFace](https://huggingface.co/wesleyacheng/news-topic-classification-with-bert).

# Setup Python Libraries

Couldn't find program: 'bash'


In [1]:
%%bash
pip freeze | grep -E '^numpy|^torch|^datasets|^transformers|^evaluate'

datasets==4.0.0
numpy==2.0.2
torch @ https://download.pytorch.org/whl/cu124/torch-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchao==0.10.0
torchaudio @ https://download.pytorch.org/whl/cu124/torchaudio-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchdata==0.11.0
torchsummary==1.5.1
torchtune==0.6.1
torchvision @ https://download.pytorch.org/whl/cu124/torchvision-0.21.0%2Bcu124-cp311-cp311-linux_x86_64.whl
transformers==4.53.3


# Create News Topic Dataset

In [2]:
from datasets import load_dataset

# let's get the news dataset from HuggingFace Hub
# https://huggingface.co/datasets/ag_news
raw_datasets = load_dataset('ag_news')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [4]:
# let's set a seed for reproducibility
SEED = 42

In [5]:
# let's create a validation set (10%) for finding the model with best generalizability
train_validation_datasets = raw_datasets['train'].train_test_split(test_size=0.1, seed=SEED)
train_validation_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 108000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12000
    })
})

In [6]:
from datasets import DatasetDict

# create datasets for fine-tuning
datasets = DatasetDict({
    'train': train_validation_datasets['train'],
    'validation': train_validation_datasets['test'],
    'test': raw_datasets['test']
})
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 108000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 12000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [7]:
# get class label information from dataset
num_labels = datasets['train'].features['label'].num_classes
id2label = {}
label2id = {}
for label_id,label in enumerate(datasets['train'].features['label'].names):
    id2label[label_id] = label
    label2id[label] = label_id

print(f"NUM_LABELS: {num_labels}")
print(f"ID2LABEL: {id2label}")
print(f"LABEL2ID: {label2id}")

NUM_LABELS: 4
ID2LABEL: {0: 'World', 1: 'Sports', 2: 'Business', 3: 'Sci/Tech'}
LABEL2ID: {'World': 0, 'Sports': 1, 'Business': 2, 'Sci/Tech': 3}


In [8]:
import random
random.seed(SEED)

# get random integers in the range of 0 to train_dataset_length
EXAMPLE_INDICES = [random.randrange(len(datasets['train'])) for _ in range(3)]
EXAMPLE_INDICES

[83810, 14592, 3278]

In [9]:
for i in EXAMPLE_INDICES:
    text = datasets['train']['text'][i]
    label_id = datasets['train']['label'][i]
    label = id2label[label_id]
    print(f" TEXT[{i}]: {text}")
    print(f"LABEL[{i}]: {label} ({label_id})")
    print()

 TEXT[83810]: Hopes fade for China mine victims Rescuers express mounting fears for 86 people still missing after a gas explosion in a China coal mine killed 62.
LABEL[83810]: World (0)

 TEXT[14592]: Notebook: Breeders #39; Cup gives title hope to several The landscape has changed since Smarty Jones #39; meteoric rise from obscurity to stardom last spring, when the colt from Pennsylvania appeared to be 
LABEL[14592]: Sports (1)

 TEXT[3278]: Max Table size in MySQL \\It turns out that MySQL  has a terrible upper limit on the max MyISAM table\size.  It appears that innodb doesn't have this problem.\\Jeremy talks about it over on his blog \\"When this happens, the first reaction I hear is "You never told me that MySQL\has a 4GB limit! What am I going to do?" Amusingly, I usually do describe the\limit when I discuss the possibility of using MySQL with various groups--they\often forget or underestimate the impact it will have. Putting that aside, the\problem is easily fixed, as that page 

In [10]:
from transformers import AutoTokenizer

# get fast BERT tokenizer
checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
# tokenize and truncate dataset by batch, while removing unused column
tokenized_datasets = datasets.map(lambda x: tokenizer(x['text'], truncation=True), batched=True, remove_columns=['text'])
# rename for multiclass fine-tuning
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')
# set format to pytorch
tokenized_datasets.set_format(type='torch')

Map:   0%|          | 0/108000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [12]:
# there will be 3 columns
# labels: the true class label that corresponds to this news
# input_ids: the token ids based on the tokenizer
# attention_mask: the attention boolean map of token ids
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 108000
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 12000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 7600
    })
})

In [13]:
for i in EXAMPLE_INDICES:
    text = datasets['train']['text'][i]
    tokenized_text = tokenized_datasets['train']['input_ids'][i]
    label_id = tokenized_datasets['train']['labels'][i].item()
    label = id2label[label_id]
    print(f"          TEXT[{i}]: {text}")
    print(f"TOKENIZED_TEXT[{i}]: {tokenized_text}")
    print(f"         LABEL[{i}]: {label_id} ({label})")
    print()

          TEXT[83810]: Hopes fade for China mine victims Rescuers express mounting fears for 86 people still missing after a gas explosion in a China coal mine killed 62.
TOKENIZED_TEXT[83810]: tensor([  101,  8069, 12985,  2005,  2859,  3067,  5694,  5343,  2869,  4671,
        15986, 10069,  2005,  6564,  2111,  2145,  4394,  2044,  1037,  3806,
         7738,  1999,  1037,  2859,  5317,  3067,  2730,  5786,  1012,   102])
         LABEL[83810]: 0 (World)

          TEXT[14592]: Notebook: Breeders #39; Cup gives title hope to several The landscape has changed since Smarty Jones #39; meteoric rise from obscurity to stardom last spring, when the colt from Pennsylvania appeared to be 
TOKENIZED_TEXT[14592]: tensor([  101, 14960,  1024, 20823,  1001,  4464,  1025,  2452,  3957,  2516,
         3246,  2000,  2195,  1996,  5957,  2038,  2904,  2144,  6047,  2100,
         3557,  1001,  4464,  1025, 23879,  2594,  4125,  2013, 27885, 28817,
        15780,  2000,  2732,  9527,  2197,  3500, 

# Setup Dataloaders

In [14]:
import torch

# set pytorch to current seed
torch.manual_seed(SEED)

In [15]:
# get current device, best use a gpu
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [16]:
from transformers import DataCollatorWithPadding

# we use a data collator for dynamic batch padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [17]:
from torch.utils.data import DataLoader

# setup dataloaders
dataloaders = {
    'train': None,
    'validation': None,
    'test': None,
}
for dataset_type in ['train', 'validation', 'test']:
    dataloaders[dataset_type] = DataLoader(
        dataset = tokenized_datasets[dataset_type],
        batch_size = 64,
        shuffle = True,
        collate_fn = data_collator,
    )

# Setup Training Configurations

In [18]:
from transformers import AutoModelForSequenceClassification

# get model form HuggingFace Hub
# https://huggingface.co/distilbert-base-uncased
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id)

# move model to current device
model.to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [23]:
from transformers import  get_scheduler
from torch.optim import AdamW

# let's use a linear scheduler with AdamW
scheduler_name = 'linear'
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0)
num_training_epochs = 1
num_training_steps = num_training_epochs * len(dataloaders['train'])
num_warmup_steps = 0
lr_scheduler = get_scheduler(
    name = scheduler_name,
    optimizer = optimizer,
    num_training_steps = num_training_steps,
    num_warmup_steps = num_warmup_steps,
)

print(f"           SCHEDULER NAME: {scheduler_name}")
print(f"                OPTIMIZER: {optimizer.__class__}")
print(f"NUMBER OF TRAINING EPOCHS: {num_training_epochs}")
print(f" NUMBER OF TRAINING STEPS: {num_training_steps}")

           SCHEDULER NAME: linear
                OPTIMIZER: <class 'torch.optim.adamw.AdamW'>
NUMBER OF TRAINING EPOCHS: 1
 NUMBER OF TRAINING STEPS: 1688


In [25]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00


In [26]:
from tqdm.auto import tqdm
import evaluate

# let's load the metrics for training and evaluation
accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')

In [27]:
def train(model, dataloader):
    # set to train mode
    model.train()
    loss = 0
    for batch in tqdm(dataloader):
        # reset gradients
        optimizer.zero_grad()

        # get predictions
        batch = {k:v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        labels = batch['labels']

        # gradient descent
        outputs.loss.backward()
        optimizer.step()
        lr_scheduler.step()

        # accumulate metrics
        loss += outputs.loss.item()
        accuracy_metric.add_batch(predictions=predictions, references=labels)
        f1_metric.add_batch(predictions=predictions, references=labels)

    # return metrics
    loss /= len(dataloader)
    accuracy = accuracy_metric.compute()
    f1 = f1_metric.compute(average='macro')
    return {'loss':loss, **accuracy, **f1}

In [28]:
def evaluate(model, dataloader):
    # set to evaluation mode
    model.eval()
    loss = 0
    # disable gradient computation
    with torch.no_grad():
        for batch in tqdm(dataloader):
            # get predictions
            batch = {k:v.to(device) for k,v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=-1)
            labels = batch['labels']

            # accumulate metrics
            loss += outputs.loss.item()
            accuracy_metric.add_batch(predictions=predictions, references=labels)
            f1_metric.add_batch(predictions=predictions, references=labels)

    # return metrics
    loss /= len(dataloader)
    accuracy = accuracy_metric.compute()
    f1 = f1_metric.compute(average='macro')
    return {'loss':loss, **accuracy, **f1}

# Fine-Tune Model with Custom Dataset

## Let's see how an UnFine-Tuned BERT performs

In [29]:
test_metrics = evaluate(model, dataloaders['test'])
print(f"TEST ACCURACY: {test_metrics['accuracy']:.5f}", end=" ; ")
print(f"F1 (MACRO): {test_metrics['f1']:.5f}")

  0%|          | 0/119 [00:00<?, ?it/s]

TEST ACCURACY: 0.23487 ; F1 (MACRO): 0.13023


We got **22% Accuracy** and **9% F1 (Macro)**, which is quite bad.😕

![sad.jpeg](attachment:7ef30b53-05a7-4844-9462-2ce0640e4a22.jpeg)

Let's do transfer learning on BERT! 🦾

## Fine-Tune BERT

In [30]:
for epoch in range(num_training_epochs):
    train_metrics = train(model, dataloaders['train'])
    validation_metrics = evaluate(model, dataloaders['validation'])

    print(f"EPOCH {epoch+1}", end=" | ")
    print(f"TRAIN LOSS: {train_metrics['loss']:.5f}", end=" | ")
    print(f"VALIDATION LOSS: {validation_metrics['loss']:.5f}", end=" ; ")
    print(f"ACCURACY: {validation_metrics['accuracy']:.5f}", end=" ; ")
    print(f"F1 (MACRO): {validation_metrics['f1']:.5f}")

  0%|          | 0/1688 [00:00<?, ?it/s]

  0%|          | 0/188 [00:00<?, ?it/s]

EPOCH 1 | TRAIN LOSS: 0.21514 | VALIDATION LOSS: 0.16826 ; ACCURACY: 0.94308 ; F1 (MACRO): 0.94284


## Let's see how a Fine-Tuned BERT performs

In [31]:
test_metrics = evaluate(model, dataloaders['test'])
print(f"TEST ACCURACY: {test_metrics['accuracy']:.5f}", end=" ; ")
print(f"F1 (MACRO): {test_metrics['f1']:.5f}")

  0%|          | 0/119 [00:00<?, ?it/s]

TEST ACCURACY: 0.94237 ; F1 (MACRO): 0.94237


**WOAH!** We got **94% Accuracy** and **94% F1 (Macro)** in 1 epoch! 🤯

![escalated-quickly.png](attachment:15942cbf-c16e-4dc3-8054-f6e056a7d100.png)

# Let's try out some examples!

In [32]:
from transformers import pipeline

news_topic_classifier = pipeline(task='text-classification', model=model, tokenizer=tokenizer, device=torch.cuda.current_device())

Device set to use cuda:0


In [33]:
# I got a paragraph from a BBC article
# https://www.bbc.com/news/world-europe-65866880
world_news = """
Ukraine's President Volodymyr Zelensky appears to have confirmed that his country's long-awaited counter-offensive against Russia has started.
"Counter-offensive and defensive actions are taking place," he said.
"""

news_topic_classifier(world_news)

[{'label': 'World', 'score': 0.9929359555244446}]

In [34]:
# I got a paragraph from an ESPN article
# https://www.espn.com/nfl/draft2024/insider/story/_/id/37801703/2024-nfl-draft-quarterback-class-top-passers-know-big-questions
sports_news = """
The 2023 NFL draft class saw a record 12 quarterbacks selected in the first five rounds, including three inside the top five picks. The 2024 class? Expect some similarities.
"""

news_topic_classifier(sports_news)

[{'label': 'Sports', 'score': 0.9956005811691284}]

In [35]:
# I got a paragraph from a Wall Street Journal article
# https://www.wsj.com/articles/inflation-drags-eurozone-economy-into-recession-b6ff6158?mod=economy_more_pos1
business_news = """
Friday's jobs report does little to change the Federal Reserve's debate over holding rates steady this month.
But it strengthens the likelihood that officials would pair any June pause with a stronger preference to raise rates later this year.
"""

news_topic_classifier(business_news)

[{'label': 'Business', 'score': 0.9930618405342102}]

In [36]:
# I got a paragraph from a science.org article
# https://www.science.org/content/article/crops-grown-without-sunlight-could-help-feed-astronauts-bound-mars
science_news = """
According to one recent estimate, a crew of six would require an estimated 10,000 kilograms of food for the trip.
NASA—which plans to send people to Mars within 2 decades—could stuff a spacecraft with prepackaged meals and launch additional supplies to the Red Planet in advance for the voyage home.
But even that wouldn’t completely solve the problem.
"""

news_topic_classifier(science_news)

[{'label': 'Sci/Tech', 'score': 0.9902308583259583}]